# Feature Engineering


Feature engineering is the creation of new input or target features from existing features.  The objective is to create features that do a better job of representing our machine learning problem to the model.  By doing so, we aim to improve the accuracy of the model.

In fact, good feature engineering can be the difference between a really poor model and a fantastic one!


For example, consider the following data set showing property prices:

![](https://drive.google.com/uc?id=1GhVFMYaznxBNJW7Gh1C2U5CBtPCfFZye)


Whilst the data is clean, a machine learning activity would benefit from some additional processing of the data.

**What additional features would we dervive from the above data that would make it more useful for house price prediction? >>**






Perhaps we could process the data to give us the following:

![](https://drive.google.com/uc?id=1I1wY35ulGjrOSXsjxVYQNIHTgzYxl4NG)


Here we can see a number of new features derived from the existing features.  We have added the district, so we can make inferences at the district level.  We have broken the date down into year and month components, so we can look for patterns across time as well as seasonal patterns.  We have converted the units so that all size measurements are in square metres.  We have derived metrics such as the price per square metre and price per bedroom.  All of these give us additional options to choose when selecting features later.

It's important to note that this isn't about generating a higher **quantity** of features.  It's about the **quality** of the features created.  We can focus the model on the **really important, high quality features**, rather than throwing our raw data at the model and hoping for the best,

Feature engineering is not a mechanical, linear process.  We need to adapt our techniques in response to the data we see, the problem we are trying to solve and our **domain knowledge**.  We may also want to consult others to utilise their domain knowledge.  We may also decide we need more, different, or better data and spend some time sourcing that.  And once we've built our model, we may come back to feature engineering to see what we can do to improve the model performance.


Let's start looking at some of the techniques in more detail using Python code.

In [ ]:
from dasi_library import *

## Binning

Binning, (also called banding or discretisation), can be used to create **new categorical features** that group individuals based on the **value ranges** of existing features.

For example, here we create a new feature, happiness_band, by binning the happiness feature into Low, Medium and High bands:

![](https://drive.google.com/uc?id=1R486p9llSgAhamzGsgiq9D_3D6-DupV3)



The above is an example of binning on numerical features.  We can also apply binning to categorical features.  Here we bin the countries into their global region:

![](https://drive.google.com/uc?id=1_7uh1wOyc--aZ7SR6xIoLeRp571Gr4fD)

We may use binning to create new target features that we want to predict.  We may also use it to create new input features. 



### Numerical binning with Python

Let's look at how we perform binning in Python.  First, we load in a dataset containing country names and their happiness scores:

In [ ]:
df1 = readCsv("fe_binning.csv")

Here are the descriptive stats for the happiness feature.  We can see that the happiness scores are continuous, between around 2.7 and 7.6.

In [ ]:
df1.describe()

And here are the first few rows of data:

In [ ]:
df1.head()

We can bin the data into Low, Medium and High bands using the following code:

In [ ]:
# Allocate happiness to bins
df1 = appendClass(df1, "happiness_band", "happiness", bins=[0,4.614304,6.279204,10], labels=["L","M","H"])
df1.head()

The bins parameter defines the boundaries of the bins.  In this case I have chosen the 25th and 75th percentiles as the boundaries.  The labels parameter allows us to name each bin, to ensure the low and high bands contain the bottom and top quartiles.  Note there is one less label than bin boundary (because we need 4 boundaries to make 3 bins).


And we can inspect the number of individuals in each class:

In [ ]:
classDistribution(df1, "happiness_band")

### Categorical binning with Python

We can also apply binning to categorical data.  Consider the country name.  We may consider that grouping countries into broader regions could help in our model creation (reasoning that the region a country in has strong predictive powers for predicting our target feature).

Let's load a mapping table, mapping countries to their region:

In [ ]:
mapping = readCsv("country_region.csv")

And now let's join the country on the original data to the region using the mapping table.  We use "left" as the how parameter to specify that all rows in the left table (df1) are included and joined to any matching rows in the right table (mapping).  Any rows in df1 that have no match in mapping will receive a null value for the region.

In [ ]:
df1 = mergeOn(df1, mapping, 'country', 'country')
df1.head()

Let's not assume every row in our data has found a matching region.  If there was no match we will get a null.  Let's check!

In [ ]:
checkForNulls(df1)

Note that there are other ways you can perform this mapping (e.g. using a dictionary and the Pandas map function).  Your coding approach will vary according to the task in hand!

## Splitting

Splitting can be used to split up an existing feature into multiple new features. 

A really common use of splitting is breaking dates and times into their component parts.

Consider the following data, showing calls to the London Fire Brigade in response to domestic fires over a period of time:

![](https://drive.google.com/uc?id=1zcc7ZwXD6OJSUyEeeX4rk4MLS0COcB-6)


We have a date column here which could be used for analysis.  But as it stands, the date has very little information value.  However, if we split this date into year, month, day and day of week we can derive models that, for example, explore whether fires in London tend to occur on particular days of the week.

![](https://drive.google.com/uc?id=1-_pF1fZ_4Nrhxc_P65xmlBlW0Adpjrro)


This exploits our domain knowledge, that domestic fires tend to occur on particular days of the week.

Sometimes our data comes with compound strings, that is strings made up of multiple items of information.  We can see one such example in the London Fire Brigade data.  The property_type contains information about the property type (e.g. Purpose Built Flats/Maisonette) and the size (e.g. 4 to 9 storeys).  

![](https://drive.google.com/uc?id=1Y8NesckaSqCGof4y7IHaZ-c4fQOLNyKc)



Having these two pieces of information combined does not help our model!  So we can split it into two separate features:

![](https://drive.google.com/uc?id=18BooD4476udiFRMqDVt-UWbIK7iIftJu)

### Splitting date and time with Python

Let's look at date splitting in Python.  First we load some data and take a look at the first few rows:

In [ ]:
df2 = pd.read_csv("fe_splitting.csv")
df2.head()

One problem to be aware of when loading dates is that Pandas does not automatically recognise dates.  We can confirm this by checking the data types of our features:

In [ ]:
df2.dtypes

We can convert the object to a datetime type:

In [ ]:
df2["timestamp_of_call"] = convertToDateTime(df2, "timestamp_of_call")

Checking again, we can see the timestamp_of_call feature is now a date:

In [ ]:
df2.dtypes

We can now add some new features by extracting the components of the timestamp_of_call:

In [ ]:
df2 = splitFeatureDate(df2, "timestamp_of_call")
df2.head()

Again, it's always worth checking that everything has converted, by checking the non-null counts in each column match:

In [ ]:
checkForNulls(df2)

### Splitting compound strings with Python

Let's look at string splitting in Python.  Let's remind ourselves of the first few rows:

In [ ]:
df2.head()

We can see that property_type consists of two parts.  To split the data, we may infer that we should split at the hyphen.  But we need to be aware some rows may not contain a hyphen, or some rows may contain multiple hyphens.  This could cause a problem.  So let's check our data to see the unique values for property_type we need to deal with:

In [ ]:
listUnique(df2,"property_type")

Looking at these individually, we don't have multiple hyphens, but we do have cases with no hyphen, which will create null entries when we split.  We will need to deal with these later if we want to use this new feature for machine learning.

Let's do the split and confirm the result:

In [ ]:
df2 = splitFeatureOnSeparator(df2, "property_type", "-", ['property_type_type', 'property_type_size'])
df2.head()

A quick check confirms that the property_type_size column contains nulls as there was no hyphen to split on:

In [ ]:
checkForNulls(df2)

We would need to make an informed decision about what to do about these nulls as explained in the data cleansing section of the course!

## One-hot encoding

The Sklearn libraries can't build models with categorical data, so if we want to build models using categorical features, we need to convert them to numerical features.  One-hot encoding is one way of doing this.

Consider the following table of countries from our happiness dataset.  

![](https://drive.google.com/uc?id=1m5yJKsp2tfDU8gd3RyuAFntdFefUaSaV)



The region column is categorical so, to enable Sklearn to use it, we need to convert it to numerical.  We could just use a lookup table to map these regions to numbers:

![](https://drive.google.com/uc?id=1W2WFkn9yrKdv3bzcrqis1SxKNYM-S1pQ)


Then our data will look like this:

![](https://drive.google.com/uc?id=12PtkWPv6exBzg1tYFVcNs5VF1k5ApevR)


But this approach introduces a problem.  Each region has been assigned a value.  Whilst the size of the value is meaningless (it's just an id), most machine learning algorithms will infer some meaning from the size of the value.  It will infer that South Asia is "greater than" East Asia & Pacific.   This will lead to meaningless models!  So this approach is not useful.

One-hot encoding places each category on a column, as a new feature and places a 1 or 0 in the column to indicate if that category is "on" or "off" for that individual.  Here is the result of applying one-hot encoding to the above data set:

![](https://drive.google.com/uc?id=14QxdIMcwAeBVscum1Jfi-WazfOUGKei_)

We call each of these new categories "dummy variables".

 ### One-hot encoding with Python
 Now let's see how one-hot encoding can be performed with Python.  First let's load our data:

In [ ]:
df3 = readCsv("fe_one_hot.csv")
df3.head()

We use the Pandas get_dummies() function to convert the required column to dummy variables:

In [ ]:
df3 = oneHotEncode(df3, ["region"])
df3.head()

## Calculated Features

In some cases, we can create new features using calculations based on existing features.

For example, consider the following data showing the total GDP and population by country:

![](https://drive.google.com/uc?id=1GekSRX94UGg4TqpmzVwYLvzxbbgltdWh)



Both the GDP and population tend to be larger for larger countries, and using them may lead to models that are too influenced by the size of the countries.  But by dividing GDP by population, we get a new measure, gdp_per_capita, which is much more useful:

![](https://drive.google.com/uc?id=1WHQyg8iIz2z4FvLV0NxLiaW_v4Ogv4lN)

There are various calculations we may perform, generating ratios such as GDP per capita, or differences between two values or even more complex calculations.  We could also create **aggregations** by grouping data then summing, taking the mean, using min or max, and so on.

The nature of the calculations very much depends on having appropriate domain knowledge.

### Calculated features in Python
Let's load some data:

In [ ]:
df4 = readCsv("fe_calculated.csv")
df4.head()

We can easily perform a calculation on existing features to create a new feature:

In [ ]:
df4["gdp_per_capita"] = df4["gdp_usd"] / df4["population"]
df4.head()